In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad

In [95]:
def generate_random_key():
    return get_random_bytes(16)

def encryption_oracle(data):
    prefix = get_random_bytes(5 + (get_random_bytes(1)[0] % 6))
    suffix = get_random_bytes(5 + (get_random_bytes(1)[0] % 6))
    data = pad(prefix + data + suffix, AES.block_size)

    key = generate_random_key()
    use_ecb = get_random_bytes(1)[0] % 2 == 0

    if use_ecb:
        cipher = AES.new(key, AES.MODE_ECB)
        ciphertext = cipher.encrypt(data)
    else:
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        ciphertext = cipher.encrypt(data)

    return ciphertext, use_ecb

def detect_block_cipher_mode(ciphertext):
    # Check for repeated blocks to detect ECB mode
    block_size = AES.block_size
    blocks = [ciphertext[i:i + block_size] for i in range(0, len(ciphertext), block_size)]

    # Check for identical adjacent blocks
    for i in range(1, len(blocks)):
        if blocks[i] == blocks[i - 1]:
            return "ECB"

    return "CBC"

if __name__ == "__main__":
    # Example usage
    for _ in range(10):
        plaintext = b'YELLOW SUBMARINE\x04\x04\x04\x04'
        ciphertext, actual_mode = encryption_oracle(plaintext)
        detected_mode = detect_block_cipher_mode(ciphertext)

        # Convert actual_mode to a string for printing
        actual_mode_str = "ECB" if actual_mode else "CBC"

        print(f"Actual Mode: {actual_mode_str}, Detected Mode: {detected_mode}")


Actual Mode: CBC, Detected Mode: CBC
Actual Mode: CBC, Detected Mode: CBC
Actual Mode: ECB, Detected Mode: CBC
Actual Mode: ECB, Detected Mode: CBC
Actual Mode: CBC, Detected Mode: CBC
Actual Mode: CBC, Detected Mode: CBC
Actual Mode: ECB, Detected Mode: CBC
Actual Mode: ECB, Detected Mode: CBC
Actual Mode: CBC, Detected Mode: CBC
Actual Mode: ECB, Detected Mode: CBC
